In [1]:
from langchain.llms.openai import OpenAI

# gpt 3.5 사용
from langchain.chat_models import ChatOpenAI, ChatAnthropic

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system. These include Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, the definition of a planet has been debated, and there may be other celestial bodies in our solar system that could potentially be classified as planets in the future.')

In [1]:
from langchain.chat_models import ChatOpenAI


# temperature 모델의 창의성 수치를 정할수 있음
# 값이 높을수록 창의성이 증가함, 원하는 값이 등장 안할수도 있음 무작위성 증가
chat = ChatOpenAI(temperature=0.1)

In [3]:
# HumanMessage 는 사람이 보내는 메세지
# AIMessage 는 ai에 의해 보내지는 것
# SystemMessage는 시스템에 설정값을 보낼때 사용함
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]
# 변수들을 지정해서 원하는 값들 넣고 활용 
# messages = [
#     SystemMessage(content="You are a geography expert. And you only reply in {language}."),
#     AIMessage(content="Ciao, mi chiamo {name}!")
#     HumanMessage(content="What is the distance between {contry} and Thailand. Also, what is your name?")
# ]

chat.predict_messages(messages=messages)


AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [4]:
chat.predict("Hello, what is your name?")

"Hello! I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?"

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You're a teacher and present your score"),
    AIMessage(content="I am a GPT teacher!"),
    HumanMessage(content="Sir, what is 30 plus 20?")
]


chat.predict_messages(messages=messages)

AIMessage(content='30 plus 20 equals 50.')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate



# Prompt 기능 사용, 사용자가 오류됀 값을 입력하는걸 방지하기 위해, 아래처럼 두개의 단어만 넣는다
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Maxico", country_b="Korea")

# chat = ChatOpenAI()
chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and South Korea is approximately 8,900 miles (14,320 kilometers) when measured in a straight line.'

In [6]:
messages = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a geography expert. And you only reply in {language}.",
        ),
        (
            "ai",
            "Ciao, mi chiamo {name}!",
        ),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)


prompt = messages.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)


In [16]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello, How, are you")

['Hello', 'How', 'are you']

In [20]:
template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else.",
    ),
    ("human", "{question}",),],
)

prompt = template.format_messages(max_items=10, question="What are the colors?",)
result = chat.predict_messages(prompt)

# 리스트로 바꾸는 parser
p = CommaOutputParser()
p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [21]:
# 이것이 체인
chain_one = template | chat | CommaOutputParser()
# chain_two = template | chat | CommaOutputParser()
# all = chain_one | chain_two | parser 이렇게도 됌

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?",
},)

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍 작업
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingreients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [25]:
# 모르겠다면 모르겠다고 말하는걸 세팅하는게 아주 중요함
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."
        ),
        (
            "human",
            "{recipe}",
        ),
    ],
)

veg_chain = veg_chef_prompt | chat

# recipe 를 담아
final_chain = {"recipe" : chef_chain} | veg_chain


final_chain.invoke(
    {
    "cuisine":"indian",
    },
    )

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp turmeric
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (to serve)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, 1/2 tsp turmeric, cayenne pepper, salt, and pepper. Add the chicken pieces and marinate for at least 1 hour, preferably overn

AIMessageChunk(content="As a vegetarian chef, I can help you make a vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. Here's how you can modify the recipe:\n\nAlternative Ingredient:\n- Instead of chicken, you can use paneer (Indian cottage cheese) or tofu as a vegetarian protein option. Both paneer and tofu have a mild flavor that absorbs the spices well, making them great substitutes for chicken in this dish.\n\nPreparation:\n- If using paneer: Cut the paneer into bite-sized cubes and follow the same marination process as you would with the chicken. Paneer doesn't need as much time to marinate, so 30 minutes should be sufficient.\n- If using tofu: Drain the tofu well and cut it into bite-sized pieces. To give the tofu a firmer texture, you can press it between paper towels with a weight on top for about 15-20 minutes before marinating. Tofu can be more delicate, so handle it gently during the marination process.\n\nFollow the rest of the

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍 작업
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)

t.format(country="France")

'What is the capital of France'

In [2]:
# FewShot이란 모델에게 예제들을 주는것
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
                    Here is what I know:
                    Capital: Paris
                    Language: French
                    Food: Wine and Cheese
                    Currency: Euro
                    """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
                    I know this:
                    Capital: Rome
                    Language: Italian
                    Food: Pizza and Pasta
                    Currency: Euro
                    """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
                I know this:
                Capital: Athens
                Language: Greek
                Food: Souvlaki and Feta Cheese
                Currency: Euro
                """,
    },
]

chat.predict("What do you know about Franch?")

Franch is a term that is often used to refer to the French language or culture. It is a colloquial term that is sometimes used in a humorous or exaggerated way to describe things that are associated with France or French culture. It is not a formal or official term, but rather a slang term that is used in casual conversation.

'Franch is a term that is often used to refer to the French language or culture. It is a colloquial term that is sometimes used in a humorous or exaggerated way to describe things that are associated with France or French culture. It is not a formal or official term, but rather a slang term that is used in casual conversation.'

In [4]:
example_template = """
    Human: {question}
    AI: {answer}
"""

# 2개다 상관없음
example_prompt = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")
chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI: 
                I know this:
                Capital: Berlin
                Language: German
                Food: Bratwurst and Sauerkraut
                Currency: Euro

AIMessageChunk(content='AI: \n                I know this:\n                Capital: Berlin\n                Language: German\n                Food: Bratwurst and Sauerkraut\n                Currency: Euro')

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "France?",
        "answer": """
                    Here is what I know:
                    Capital: Paris
                    Language: French
                    Food: Wine and Cheese
                    Currency: Euro
                    """,
    },
    {
        "country": "Italy?",
        "answer": """
                    I know this:
                    Capital: Rome
                    Language: Italian
                    Food: Pizza and Pasta
                    Currency: Euro
                    """,
    },
    {
        "country": "Greece?",
        "answer": """
                I know this:
                Capital: Athens
                Language: Greek
                Food: Souvlaki and Feta Cheese
                Currency: Euro
                """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography exper"),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt |  chat

chain.invoke({"country":"Germany"})

I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final, 
    pipeline_prompts=prompts,
    )
# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg! That is a secret!! Arg arg!",
    "question":"What is your fav food?",
})

Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack biscuits! Aye, nothing beats the taste of the sea on me tongue! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite grub be a hearty plate of salted beef and hardtack biscuits! Aye, nothing beats the taste of the sea on me tongue! Arrr!')

In [20]:
# Caching
# 똑같은 질문을 받아도 같은 답변을 줌으로써 반복 해결
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.78s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thi

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or t

In [19]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjo

In [22]:
# 비용 계산 및 사용 토큰 계산
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of sugar\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for at least 1 hour.\n2. Drain the rice and steam it until it is fully cooked.\n3. In a large bowl, mix the cooked rice with water and nuruk.\n4. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.\n5. After fermentation, strain the mixture through a cheesecloth to remove any solids.\n6. Add sugar to the strained liquid and mix well.\n7. Transfer the liquid to a clean bottle and let it sit for another 1-2 days to allow the flavors to develop.\n8. Serve the homemade soju chilled and enjoy!",
        "generation_info": {
          "finish_

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm


chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k",
)
# 모델 저장
chat.save("model.json")

# 모델 불러오기
chat = load_llm("model.json")
chat

In [25]:
# 메모리 기억, 전체를 기억하는 타입이라 용량이 매우큼, 비효율적
from langchain.memory import ConversationBufferMemory

#챗모델이면 return messages True로 하면 대화 내용으로 나옴
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [28]:
# 대화 버퍼 윈도우 메모리
# 대화의 특정 부분만을 저장하는 메모리
# 단점은 전체대화가 아닌 최근꺼만 기억함
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

In [29]:
add_message(1, 1)

In [30]:
add_message(2, 2)

In [31]:
add_message(3, 3)

In [32]:
add_message(4, 4)

In [34]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [35]:
add_message(5, 5)

In [36]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [37]:
# SummaryMemory, LLM을 이용한 기억
# 메모리를 사용하여 요금이 좀 나감, 메모리 요약을 아주 잘해줌, 길수록 유용함
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Seunggyu, I live in South Korea", "Wow that is so cool!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman

In [38]:
add_message("South Korea is so pretty", "I wish I could go!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "The human introduces themselves as Seunggyu from South Korea. The AI responds with enthusiasm at the information.",
  "new_lines": "Human: South Korea is so pretty\nAI: I wish I could go!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach

In [39]:
get_history()

{'history': 'The human introduces themselves as Seunggyu from South Korea. The AI responds with enthusiasm at the information, expressing a desire to visit the country because it is so pretty.'}

In [40]:
# memory, 최근의 상호작용은 추적, 오래됀 메세지를 요약해서 저장
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm, max_token_limit=150,
                                   return_messages=True,)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Seunggyu, I live in South Korea", "Wow that is so cool!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman

In [41]:
get_history()

{'history': 'The human introduces themselves as Seunggyu from South Korea. The AI responds with enthusiasm at the information.'}

In [42]:
add_message("South Korea is so pretty", "I wish I could go!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "The human introduces themselves as Seunggyu from South Korea. The AI responds with enthusiasm at the information.",
  "new_lines": "Human: South Korea is so pretty\nAI: I wish I could go!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach

In [43]:
get_history()

{'history': 'The human introduces themselves as Seunggyu from South Korea. The AI responds with enthusiasm at the information, expressing a desire to visit the country because it is so pretty.'}

In [44]:
# COnversationKGMemory
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 가장 중요한것만 뽑아내는 것
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Seunggyu, I live in South Korea", "Wow that is so cool!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "history": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!",
  "input": "Hi I'm Seunggyu, I live in South Korea",
  "verbose": true
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the last line of conversation. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.\n\nEXAMPLE\nConversation history:\nPerson #1: Did you hear aliens landed in Area 51?\nAI: No, I didn't hear that. 

In [45]:
memory.load_memory_variables({"input":"who is Seunggyu"})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "history": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!",
  "input": "who is Seunggyu"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: h

{'history': [SystemMessage(content='On Seunggyu: Seunggyu lives in South Korea.')]}

In [46]:
add_message("Seunggyu likes kimchi", "Wow that is so cool!")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "history": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!\nHuman: Seunggyu likes kimchi\nAI: Wow that is so cool!",
  "input": "Seunggyu likes kimchi",
  "verbose": true
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the last line of conversation. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.\n\nEXAMPLE\nConversation history:\nPerson #1: Did you hear aliens landed in

In [47]:
memory.load_memory_variables({"input":"what does Seunggyu like"})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "history": "Human: Hi I'm Seunggyu, I live in South Korea\nAI: Wow that is so cool!\nHuman: Seunggyu likes kimchi\nAI: Wow that is so cool!",
  "input": "what does Seunggyu like"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. \"What do you know about him\" where \"him\" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simpl

{'history': [SystemMessage(content='On Seunggyu: Seunggyu lives in South Korea. Seunggyu likes kimchi.')]}

In [54]:
# memory on Chain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)
# template 안에랑 이름은 일치해야함
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    YOu:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    # prompt=PromptTemplate.from_template("{question}"),
    prompt=PromptTemplate.from_template(template=template),
    verbose=True,
)

chain.predict(question="My name is Seunggyu")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Seunggyu
    YOu:


> Finished chain.


'Hello Seunggyu! How can I assist you today?'

In [55]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Seunggyu
AI: Hello Seunggyu! How can I assist you today?
    Human:I live in Seoul
    YOu:


> Finished chain.


"That's great to know! How can I assist you with living in Seoul today?"

In [56]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Seunggyu
AI: Hello Seunggyu! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with living in Seoul today?
    Human:What is my name?
    YOu:


> Finished chain.


'Your name is Seunggyu.'

In [57]:
memory.load_memory_variables({})

{'chat_history': "Human: My name is Seunggyu\nAI: Hello Seunggyu! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to know! How can I assist you with living in Seoul today?\nHuman: What is my name?\nAI: Your name is Seunggyu."}

# Chat 베이스 메모리

In [60]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Seunggyu")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Seunggyu

> Finished chain.


'Nice to meet you, Seunggyu! How can I assist you today?'

In [61]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Seunggyu
AI: Nice to meet you, Seunggyu! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'

In [62]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Seunggyu
AI: Nice to meet you, Seunggyu! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


'Your name is Seunggyu.'

# LCEL Based Memory

In [71]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context({"input":question}, {"output": result.content})
    print(result)

In [72]:
invoke_chain("My name is Seunggyu")

{'question': 'My name is Seunggyu'}
content='Nice to meet you, Seunggyu! How can I assist you today?'


In [73]:
invoke_chain("What is my name?")

{'question': 'What is my name?'}
content='Your name is Seunggyu.'
